In [2]:
import sys
import os

# Add the path to the aqi package
aqi_path = os.path.abspath('../aqi')
sys.path.append(aqi_path)

from fetch import fetch_data
from etl import validate_data, parse_data, clean_data, push_to_db

In [3]:
aqi_data = fetch_data()
print(aqi_data['status'])
print(len(aqi_data['data']))

<Response [200]>
{'status': 'ok', 'data': [{'lat': 13.917563, 'lon': 101.590529, 'uid': 9014, 'aqi': '97', 'station': {'name': 'Tha Tum, Si Maha Phot, Prachin Buri, Thailand (ต.ท่าตูม อ.ศรีมหาโพธิ จ.ปราจีนบุรี)', 'time': '2025-02-22T21:00:00+09:00'}}, {'lat': 16.8256, 'lon': 96.1445, 'uid': 11784, 'aqi': '109', 'station': {'name': 'Rangoon US Embassy, Maynmar', 'time': '2025-02-22T20:30:00+09:00'}}, {'lat': 7.77355528011, 'lon': 98.32379579544, 'uid': -501961, 'aqi': '19', 'station': {'name': 'Wiset Road', 'time': '2025-02-22T22:00:01+09:00'}}, {'lat': 10.008541, 'lon': 99.066484, 'uid': -419677, 'aqi': '-', 'station': {'name': 'ขวัญเมือง เอนเนอร์จี', 'time': '2025-02-22T20:40:03+09:00'}}, {'lat': 15.820324, 'lon': 102.059997, 'uid': -419323, 'aqi': '38', 'station': {'name': 'คาลเท็กซ์ บายพาสชัยภูมิ (ขวัญไทยปิโตรเลียม)', 'time': '2025-02-22T21:49:09+09:00'}}, {'lat': 5.629631, 'lon': 100.467771, 'uid': 2582, 'aqi': '45', 'station': {'name': 'Bakar Arang, Sg. Petani, Kedah, Malaysia', '

In [4]:
validate_data(aqi_data)

In [5]:
parse_aqi_data = parse_data(aqi_data)
print(parse_aqi_data[:3])

      uid  aqi   latitude   longitude  \
0    9014   97  13.917563  101.590529   
1   11784  109  16.825600   96.144500   
2 -501961   19   7.773555   98.323796   

                                        station_name                 timestamp  
0  Tha Tum, Si Maha Phot, Prachin Buri, Thailand ... 2025-02-22 21:00:00+09:00  
1                        Rangoon US Embassy, Maynmar 2025-02-22 20:30:00+09:00  
2                                         Wiset Road 2025-02-22 22:00:01+09:00  


In [6]:
clean_aqi_data = clean_data(parse_aqi_data)
print(clean_aqi_data[:3])

      uid    aqi   latitude   longitude  \
0    9014   97.0  13.917563  101.590529   
1   11784  109.0  16.825600   96.144500   
2 -501961   19.0   7.773555   98.323796   

                                        station_name                 timestamp  
0  Tha Tum, Si Maha Phot, Prachin Buri, Thailand ... 2025-02-22 21:00:00+09:00  
1                        Rangoon US Embassy, Maynmar 2025-02-22 20:30:00+09:00  
2                                         Wiset Road 2025-02-22 22:00:01+09:00  


In [8]:
import psycopg2

# Database connection parameters
db_params = {
    'dbname': 'air_data',
    'user': 'admin',
    'password': 'adminpassword',
    'host': 'localhost',
    'port': '54329'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Push the cleaned AQI data to the database
push_data = push_to_db(conn, clean_aqi_data)

# Close the connection
conn.close()